# Altair Practice Lab

In this assignment we will be crafting a series of visualizations using
Altair to get practice working with real data in this context.

Your responses should be within the functions given,
using appropriate helper functions to help with clarity
and reduce redundancy.

## cd Rubric

The criteria to receive an S on this assignment is a good-faith attempt at each portion.

A good-faith attempt should either:

- be fundamentally correct producing the expected output with minimal deviations,
- **OR** contain an explanation of what does not work and _details on what was tried_.

**To receive an N, at least half of the assignment should have good-faith attempts.**

Your charts do not need to match the examples exactly!  They are helpful to get a sense of what you're after, but focus on the problem description.

## Introducing the Dataset

In the data directory you'll find three files:

**legislators.csv** which consists of ~7400 records representing state legislators, it has the following fields:

- name
- given_name
- family_name
- party: As reported by the state.
- gender: Male / Female / Other\*
- jurisdiction: This field contains an identifier for the state or jurisdiction, see below for details.
- district: the name of the district represented
- type: upper | lower - The classification of the legislative. Most states have both, but DC and NE only have an upper chamber.

Note: Accurate data on gender is hard to come by in many states. There may be irregularities in this field. This is also why this field does not make further distinctions beyond Male/Female/Other.

**populations.csv** 

A CSV file with a row with each state's population.  

**actions.csv**

A CSV file with legislative actions for a few states across a range of years:

- description: action text as reported by state
- date: date of action recorded
- classification: not used for this assignment
- state: 2-letter state postal code (note: these are upper case, other files use lower case)
- session: an identifier representing which session the action comes from (not used)

In [406]:
# PUT YOUR IMPORTS HERE
import pandas as pd 
import altair as alt
import numpy as np

## Part 1: Data Exploration

First, we'll build a few exploratory visualizations to get a sense of the data for this assignment.

### 1.0: Cleaning

As mentioned above, there is no 'state' field. Jurisdiction is in the format:

`ocd-jurisdiction/country:us/state:nc/government`

and for non-states:

- `ocd-jurisdiction/country:us/district:dc/government`
- `ocd-jurisdiction/country:us/territory:pr/government`

So for our purposes, we want to add a `state` column from the two letter code after either "state:", "district:", or "territory:".
(We will treat DC and PR as states.)

Complete the function `legislators_df` which should return the data from `legislators.csv` in a dataframe, with an additional `state` column.

In [407]:
def legislators_df():

    legislators_df = pd.read_csv('data/legislators.csv')
    legislators_df['state'] = legislators_df['jurisdiction'].str[-13:-11].str.upper()

    return legislators_df
    
# render dataframe
legislators_df()

,name,given_name,family_name,gender,party,jurisdiction,district,type,state
0,Robert White,Robert C.,White,Male,Independent,ocd-jurisdiction/country:us/district:dc/govern...,At-Large,legislature,DC
1,Charles Allen,Charles,Allen,Male,Democratic,ocd-jurisdiction/country:us/district:dc/govern...,Ward 6,legislature,DC
2,Vince Gray,Vincent Condol,Gray,Male,Democratic,ocd-jurisdiction/country:us/district:dc/govern...,Ward 7,legislature,DC
3,Phil Mendelson,Philip Heath,Mendelson,Male,Democratic,ocd-jurisdiction/country:us/district:dc/govern...,Chairman,legislature,DC
4,Trayon White,Trayon,White,Male,Democratic,ocd-jurisdiction/country:us/district:dc/govern...,Ward 8,legislature,DC
...,...,...,...,...,...,...,...,...,...
7430,Sol Higgins,Sol Yamiz,Higgins Cuadrado,Female,Partido Popular Democrático,ocd-jurisdiction/country:us/territory:pr/gover...,35,lower,PR
7431,Migdalia González,Migdalia I.,Gonzalez Arroyo,Female,Partido Popular Democrático,ocd-jurisdiction/country:us/territory:pr/gover...,4,upper,PR
7432,Keren Riquelme,Keren L.,Riquelme Cabrera,Female,Partido Nuevo Progresista,ocd-jurisdiction/country:us/territory:pr/gover...,At-Large,upper,PR
7433,Jocelyne Rodríguez Negrón,Jocelyne M.,Rodriguez Negron,Female,Partido Popular Democrático,ocd-jurisdiction/country:us/territory:pr/gover...,19,lower,PR


### 1.1: Initial Plot

First let's build a visualization of gender breakdowns in state legislatures.
Use the following:

- stacked bars per state
- each segment of stacked bar is gender

Your graph should somewhat resemble *imgs/ex1.1.png*.

In [185]:
def states_by_gender_initial(df):

    print(df.shape[0])
    df['count'] = 1
    gender_groupby = df.groupby(['state','gender'])['count'].sum()
    gender_df = gender_groupby.reset_index()
    gender_breakdown = alt.Chart(gender_df).mark_bar().encode(alt.X('state').title('State'), alt.Y('count').title('Record Count'),color='gender')
    
    return gender_breakdown
    
# render chart
states_by_gender_initial(legislators_df())

7435


alt.Chart(...)

### 1.2: Improvements

While it is clear from the first chart that there are more elected officials that are men than women, it is hard to compare across states.

Make the following adjustments:

- Normalize the chart so that each bar is a percentage, allowing for direct comparison across states.
- Since this is US political data, the colors red and blue have a strong meaning, associated with the Republican and Democratic parties. Change the color scheme to avoid red and blue. (I chose #8624f5 for women and #1fc3aa for men based on this article: <https://blog.datawrapper.de/gendercolor/>)
- Two states are very close to 50%, add a line at 50% using a layered chart to make it easier to see if they exceed 50% or not.

Your graph should somewhat resemble *imgs/ex1.2.png*.

In [350]:
def states_by_gender_improved(df):

    df['count'] = 1
    gender_df = df.groupby(['state','gender'])['count'].sum().reset_index()
    gender_totals = gender_df.groupby(['state'])['count'].sum().reset_index('state').rename(columns={'count':'total'})
    gender_df = pd.merge(gender_df, gender_totals, how='left',on=['state'])
    gender_df['%'] = gender_df['count']/gender_df['total']
    
    source = gender_df
    
    base = alt.Chart(source)
    range_ = ['#8624f5','#1fc3aa','gray']
    gender_breakdown = base.mark_bar().encode(alt.X('state').title('State'),alt.Y('%').title('Count of Records'),color=alt.Color('gender').scale(range=range_))

    line = alt.Chart(pd.DataFrame({'y':[0.50]})).mark_rule(color='black').encode(y='y')

    chart = gender_breakdown + line
    return chart
    
# render chart
states_by_gender_improved(legislators_df())

alt.LayerChart(...)

## Part 2: Party Breakdown

We'll now take a look at party control. We can start with essentially the same chart.

### 2.0 - Party Control

Copy your code from 1.2 above & modify it to use party instead of gender.  Your graph will wind up with too many parties, see `imgs/ex2.0.png`.

In [351]:
def party_control_raw(df):
    df['count'] = 1
    party_df = df.groupby(['state','party'])['count'].sum().reset_index()
    party_totals = party_df.groupby(['state'])['count'].sum().reset_index('state').rename(columns={'count':'total'})
    party_df = pd.merge(party_df, party_totals, how='left',on=['state'])
    party_df['%'] = party_df['count']/party_df['total']
    
    base = alt.Chart(party_df)
    range_ = ['#8624f5','#1fc3aa','gray']
    party_breakdown = base.mark_bar().encode(alt.X('state').title('State'),alt.Y('%').title('Count of Records'),color=alt.Color('party').scale(range=range_))

    line = alt.Chart(pd.DataFrame({'y':[0.50]})).mark_rule(color='black').encode(y='y')

    chart = party_breakdown + line
    return chart
    
# render chart
party_control_raw(legislators_df())

alt.LayerChart(...)

### 2.1 - Cleaning Data

The above graph still has some shortcomings:

- Most states have an upper and lower chamber, and party control may vary between them. We'll need to make two bars per state (which we'll tackle in 2.2).
- Also, there are too many variations of party as you can see here:

Let's transform the data again, adding a new column "party_code" with the following rules:

- if the word 'Democratic' appears, set party_code to 'D'
- if the word 'Republican' appears, set the party_code to 'R'
- otherwise, set the party_code to 'O'

Party data in NE, DC, and PR does not work with this scheme.
For simplicity, we will exclude them from our analysis.

For this portion, implement `clean_party_df` which should return a modified legislators DataFrame with the `party_code` column, and the rows for states 'DC', 'NE' and 'PR' dropped.

In [352]:
def clean_party_df():
    
    df = legislators_df()
    
    # drop states with alternative party scheme 
    df = df.loc[df['state'].isin(['DC','NE','PR'])==False]

    df['party_code'] = np.where(df['party'].str.contains('Democratic')==True, 'D',np.where(df['party'].str.contains('Republican')==True,'R','O'))

    return df

clean_party_df()

,name,given_name,family_name,gender,party,jurisdiction,district,type,state,party_code
13,Kevin McCabe,Kevin J.,McCabe,Male,Republican,ocd-jurisdiction/country:us/state:ak/government,30,lower,AK,R
14,C.J. McCormick,Conrad J.,McCormick,Male,Democratic,ocd-jurisdiction/country:us/state:ak/government,38,lower,AK,D
15,Bill Wielechowski,Bill P.,Wielechowski,Male,Democratic,ocd-jurisdiction/country:us/state:ak/government,K,upper,AK,D
16,Justin Ruffridge,Justin,Ruffridge,Male,Republican,ocd-jurisdiction/country:us/state:ak/government,7,lower,AK,R
17,Will Stapp,Will,Stapp,Male,Republican,ocd-jurisdiction/country:us/state:ak/government,32,lower,AK,R
...,...,...,...,...,...,...,...,...,...,...
7353,Karlee Provenza,Karlee R.,Provenza,Female,Democratic,ocd-jurisdiction/country:us/state:wy/government,45,lower,WY,D
7354,Affie Ellis,Affie Burnside,Ellis,Female,Republican,ocd-jurisdiction/country:us/state:wy/government,8,upper,WY,R
7355,Abby Angelos,Abby,Angelos,Female,Republican,ocd-jurisdiction/country:us/state:wy/government,3,lower,WY,R
7356,Pepper Ottman,Pepper L.,Ottman,Female,Republican,ocd-jurisdiction/country:us/state:wy/government,34,lower,WY,R


### 2.2 - Faceted Plot

Add a function `party_control_by_chamber` that contains the following elements:

- One bar per state, **along the Y axis**.
- Each bar should consist of a stack: a blue portion, a green portion, and a red portion, corresponding to the D, O, and R `party_code` respectively.
- A vertical line at the 50% mark, indicating (likely) party control.
- Finally, facet the chart on `type` so that you get a set of bars for the lower and upper chambers.

See `imgs/ex2.2.png` for an example.


In [402]:
def party_control_by_chamber(df):

    df['count'] = 1
    party_control_df = df.groupby(['state','type','party_code'])['count'].sum().reset_index()
    party_control_totals_df = party_control_df.groupby(['state','type'])['count'].sum().reset_index().rename(columns={'count':'total'})
    party_control_df = pd.merge(party_control_df,party_control_totals_df, how='left',on=['state','type'])
    party_control_df['%'] = party_control_df['count']/party_control_df['total']*100
    party_control_df_perc_total = party_control_df.groupby(['state','type'])['%'].sum().reset_index()
    party_control_df['50'] = 50
    source = party_control_df
    base = alt.Chart(source)
    party_control = base.mark_bar().encode(alt.X('%',title='Percent'),alt.Y('state'), color='party_code')
    # line = alt.Chart(pd.DataFrame({'x':[50.0]})).mark_rule().encode(x='x')
    line = base.mark_rule().encode(x='50')
    chart = (party_control + line).facet(column='type')
    return chart

    
party_control_by_chamber(clean_party_df())

alt.FacetChart(...)

## Part 3: Comparing by Population

For part three, we are interested in the relationship of various properties of legislatures to the total population of the state.

To do this, we'll need to create a combined DataFrame that mixes in data from `populations.csv`.

### 3.0 - Create Combined DataFrame

Write the function `population_combined_df`, which should return a DataFrame with the columns:

- state: abbreviation of state
- upper: total seats in upper chamber
- lower: total seats in lower chamber
- pop_2020: the 2020 population, obtained from merging with `population.csv`

**Data Note:** These numbers are based on the non-vacant seats as-of a particular day in September 2024. Vacancies will cause the counts to be off by a bit, but the general size of the legislature should be roughly the same.

In [492]:
def population_combined_df():

    seats_df = legislators_df()
    seats_df = seats_df.groupby(['state','type']).agg({'type':'size'}).unstack().reset_index()
    seats_df.columns = ['state','legislature','lower','upper']
    seats_df = seats_df[['state','lower','upper']]
    
    pop_df = pd.read_csv('data/populations.csv')
    pop_df['state'] = pop_df['state'].str.upper()
    
    seats_df = pd.merge(seats_df,pop_df, how='left',on=['state'])
    
    return seats_df
    

population_combined_df()

,state,lower,upper,pop_2020
0,AK,40.0,20.0,733374
1,AL,104.0,35.0,5024294
2,AR,98.0,35.0,3011490
3,AZ,59.0,30.0,7157902
4,CA,79.0,40.0,39538212
5,CO,65.0,35.0,5773707
6,CT,151.0,36.0,3605912
7,DC,NaN,NaN,689548
8,DE,41.0,21.0,989946
9,FL,119.0,40.0,21538216


### 3.1 - Create Population vs. Seats Scatterplot

Create a new plot with two layers:

- Population on the X axis
- Number of seats on the Y axis
- Upper chamber points should be purple and use the 'triangle-up' shape.
- Lower chamber points should be orange and use the 'triangle-down' shape.
- Make a customization or two to your chart's default labels and axes, whatever you feel is appropriate.

Hint: You can layer two charts for this.

In [524]:
def scatter_pop_size():
    source = population_combined_df()
    lower_chart = alt.Chart(source).mark_point(color='orange',shape='triangle-down').encode(alt.X('pop_2020'),alt.Y('lower'))
    upper_chart = alt.Chart(source).mark_point(color='purple',shape='triangle-up').encode(alt.X('pop_2020'),alt.Y('upper'))
    chart = lower_chart + upper_chart
    return chart

scatter_pop_size()

alt.LayerChart(...)

### 3.2 - Regressions

Add two more layers, a purple & orange regression line for each chamber.  See `imgs/ex3.2.png`

Hint: See `transform_regression`.

In [43]:
def scatter_pop_size_regression():
    pass # IMPLEMENTATION HERE

## Part 4: Actions Heatmap

The file `actions_il-in-mi-wi_2021-2024.csv` contains nearly half a million records, representing every official action taken on a piece of legislation in these four midwestern states over the past two sessions.

Legislatures work quite differently, some meet all year, while others meet for very short periods.
By creating a heatmap of what days actions take place, we can get a sense of how different states compare.

### 4.0 - Load Actions

Complete `actions_df`, which should load the data from `actions_il-in-mi-wi_2021-2024.csv`.

Tips: 
- Make sure that the `date` column is loaded as a date type!
- Dates are in YYYY-MM-DD format, though some also have additional characters for time, which you will want to ignore.

In [36]:
def actions_df():
    pass # IMPLEMENTATION HERE

### 4.1 - Actions Heatmap

Generate a heatmap (using `mark_rect`) with:

- a row per state
- each row consists of shaded marks with shading based on the total action count for a given week

Tip: Use the 'yearweek(date)' aggregation for the X channel.

See `imgs/ex4.1.png`.

In [37]:
def actions_heatmap_scaled(df):
    pass # IMPLEMENTATION HERE

actions_heatmap_scaled(actions_df())

### 4.2 - Excluding IL Outliers

Illinois clearly dominates the above graph, below modify two calls to `actions_heatmap` with a modified dataframe that excludes IL, and a modified dataframe that only includes IL.

(Note how by using functions in our dataframe we can more easily reuse portions by making small adjustments to the data.)

See `ex4.2a.png` and `ex4.2b.png`

In [39]:
# MODIFY THIS LINE (exclude IL)
actions_heatmap_scaled(actions_df())

In [40]:
# MODIFY THIS LINE (only IL)
actions_heatmap_scaled(actions_df())

#### 4.3 - Cumulative Line Chart

Another way to view this data would be with a cumulative line chart.

Create a chart with:

- days on the X axis
- cumulative actions to date on the Y axis
- one line per state

Hint: To do this you will need to look at the `transform_window` function.

See `ex4.3.png` for an example.

In [45]:
def actions_cumulative(df):
    pass # IMPLEMENTATION HERE

actions_cumulative(actions_df())